In [141]:
import pandas as pd
 
train_set = pd.read_table('train.tsv')
train_set.columns = ['id','label','statement','subjects','speaker','speakers job title','state info','party aff','barely true counts','false counts','half true counts','mostly true counts','pants on fire counts','venue']
train_set.head()

,id,label,statement,subjects,speaker,speakers job title,state info,party aff,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,venue
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [142]:
train_set = train_set.drop(['id','subjects','speaker','speakers job title','state info','party aff','barely true counts','false counts','half true counts','mostly true counts','pants on fire counts','venue'], axis=1)
train_set.head()

,label,statement
0,half-true,When did the decline of coal start? It started...
1,mostly-true,"Hillary Clinton agrees with John McCain ""by vo..."
2,false,Health care reform legislation is likely to ma...
3,half-true,The economic turnaround started at the end of ...
4,true,The Chicago Bears have had more starting quart...


In [143]:
print(train_set['label'].value_counts())
print(train_set.shape)
train_set.isnull().sum()

half-true      2114
false          1994
mostly-true    1962
true           1676
barely-true    1654
pants-fire      839
Name: label, dtype: int64
(10239, 2)


label        0
statement    0
dtype: int64

In [144]:
for i in range(len(train_set)):
    if (train_set['label'][i] == 'mostly-true'):
        train_set['label'][i]='True'
    elif(train_set['label'][i] == 'true'):
        train_set['label'][i]='True'
    elif(train_set['label'][i] == 'half-true'):
        train_set['label'][i]='True'
    else :
        train_set['label'][i]='False'

train_set.head()


,label,statement
0,True,When did the decline of coal start? It started...
1,True,"Hillary Clinton agrees with John McCain ""by vo..."
2,False,Health care reform legislation is likely to ma...
3,True,The economic turnaround started at the end of ...
4,True,The Chicago Bears have had more starting quart...


In [145]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

stpwrds = list(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
for x in range(len(train_set)) :
    corpus = []
    review = train_set['statement'][x]
    review = re.sub(r'[^a-zA-Z\s]', '', review)
    review = review.lower()
    review = nltk.word_tokenize(review)
    for y in review :
        if y not in stpwrds :
            corpus.append(lemmatizer.lemmatize(y))
    review = ' '.join(corpus)
    train_set['statement'][x] = review 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [146]:
train_set.head()

,label,statement
0,True,decline coal start started natural gas took st...
1,True,hillary clinton agrees john mccain voting give...
2,False,health care reform legislation likely mandate ...
3,True,economic turnaround started end term
4,True,chicago bear starting quarterback last year to...


In [147]:
x = train_set.iloc[:,1]
y = train_set.iloc[:,0]
y.head()

0     True
1     True
2    False
3     True
4     True
Name: label, dtype: object

In [148]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20)

In [149]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer( min_df=2, max_df=0.95, max_features = 200000, ngram_range = ( 1, 4 ),sublinear_tf = True )

vectorizer = vectorizer.fit(x_train)
tfidf_x_train = vectorizer.transform(x_train)
tfidf_x_test = vectorizer.transform(x_test)
tfidf_x_train.shape

(8191, 14831)

In [150]:
from sklearn.linear_model import LogisticRegression

m1 = LogisticRegression()
m1.fit(tfidf_x_train,y_train)

ypred_m1 = m1.predict(tfidf_x_test)
d1 = pd.DataFrame({'y_test':y_test,'ypred_m1':ypred_m1})
d1.head()

,y_test,ypred_m1
3261,False,False
4049,False,True
8459,False,True
1075,False,False
1914,True,True


In [151]:
from sklearn.metrics import classification_report,accuracy_score

print('\nClassification Report :-\n',classification_report(y_test,ypred_m1))
print('\nAccuracy Score : ',accuracy_score(y_test,ypred_m1))


Classification Report :-
               precision    recall  f1-score   support

       False       0.58      0.43      0.50       893
        True       0.63      0.76      0.69      1155

    accuracy                           0.62      2048
   macro avg       0.61      0.60      0.59      2048
weighted avg       0.61      0.62      0.61      2048


Accuracy Score :  0.6171875


In [152]:
import numpy as np
import pandas as pd
import os
import time
import gc
import random
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing import text, sequence
from keras_preprocessing.sequence import pad_sequences
import torch
import torch.nn as nn
from torch.utils import data
from torch.nn import functional as F

In [153]:
test = pd.read_table('test.tsv')
test.columns = ['id','label','statement','subjects','speaker','speakers job title','state info','party aff','barely true counts','false counts','half true counts','mostly true counts','pants on fire counts','venue']

val = pd.read_table('valid.tsv')
val.columns= ['id','label','statement','subjects','speaker','speakers job title','state info','party aff','barely true counts','false counts','half true counts','mostly true counts','pants on fire counts','venue']


In [154]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [155]:
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 220

In [156]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def train_model(model, train, val ,test, loss_fn, output_dim, lr=0.001, batch_size=512, n_epochs=4):
    param_lrs = [{'params': param, 'lr': lr} for param in model.parameters()]
    optimizer = torch.optim.Adam(param_lrs, lr=lr)

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.6 ** epoch)
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=False)
    
    all_test_preds = []
    
    for epoch in range(n_epochs):
        start_time = time.time()
        
        scheduler.step()
        model.train()
        avg_loss = 0.
        
        
        for data in train_loader:
            x_batch = data[:-1]
            y_batch = data[-1]

            y_pred = model(*x_batch)  
            y_pred=y_pred.squeeze()
            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
        model.train()
        avg_loss = 0.
        
        
        for data in train_loader:
            x_batch = data[:-1]
            y_batch = data[-1]

            y_pred = model(*x_batch)  
            y_pred=y_pred.squeeze()
            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()


        model.eval()
        val_preds = np.zeros((len(val), output_dim))
        for data in val_loader:
            x_batch = data[:-1]
            y_batch = data[-1]

            y_pred = model(*x_batch)  
            y_pred=y_pred.squeeze()
            loss = loss_fn(y_pred, y_batch)          
            avg_loss += loss.item() / len(train_loader)
        
        elapsed_time = time.time() - start_time
        print('Epoch {}/{} \t loss={:.4f} \t time={:.2f}s'.format(epoch + 1, n_epochs, avg_loss, elapsed_time))

    model.eval()
    test_preds = np.zeros((len(test), output_dim))
     
    for i, x_batch in enumerate(test_loader):
        y_pred = sigmoid(model(*x_batch).detach().cpu().numpy())
        test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred

    all_test_preds.append(test_preds)
        
    return test_preds

In [157]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in f)

def build_matrix(word_index,path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [158]:
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix):
        super(NeuralNet, self).__init__()
        embed_size = embedding_matrix.shape[1]
        
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False

        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)

        
    def forward(self, x):
        h_embedding = self.embedding(x)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        avg_pool = torch.mean(h_lstm2, 1)

        max_pool, _ = torch.max(h_lstm2, 1)
        
        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_conc))
        h_conc_linear2  = F.relu(self.linear2(h_conc))
        
        hidden = h_conc + h_conc_linear1 + h_conc_linear2
        
        result = self.linear_out(hidden)
        
        return result

In [159]:
def preprocess(data):

    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
    return data

In [160]:
pd.unique(train_set.label)

array(['True', 'False'], dtype=object)

In [161]:
binary_dict = {'False':0 , 'True':1}

In [162]:
from nltk.inference.prover9 import test_config
for i in range(len(test)):
    if (test['label'][i] == 'mostly-true'):
        test['label'][i]='True'
    elif(test['label'][i] == 'true'):
        test['label'][i]='True'
    elif(test['label'][i] == 'half-true'):
        test['label'][i]='True'
    else :
        test['label'][i]='False'

test.head()

<ipython-input-162-672e879475ad>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'][i]='False'
<ipython-input-162-672e879475ad>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'][i]='True'
<ipython-input-162-672e879475ad>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'][i]='True'
<ipython-input-162-672e879475ad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,id,label,statement,subjects,speaker,speakers job title,state info,party aff,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,venue
0,11685.json,False,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference
1,11096.json,False,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.
2,5209.json,True,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show
3,9524.json,False,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video
4,5962.json,True,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",brendan-doherty,NaN,Rhode Island,republican,1,2,1,1,0,a campaign website


In [163]:
for i in range(len(val)):
    if (val['label'][i] == 'mostly-true'):
        val['label'][i]='True'
    elif(val['label'][i] == 'true'):
        val['label'][i]='True'
    elif(val['label'][i] == 'half-true'):
        val['label'][i]='True'
    else :
        val['label'][i]='False'

val.head()

<ipython-input-163-046aa21ed480>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['label'][i]='False'
<ipython-input-163-046aa21ed480>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['label'][i]='True'
<ipython-input-163-046aa21ed480>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['label'][i]='True'
<ipython-input-163-046aa21ed480>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,id,label,statement,subjects,speaker,speakers job title,state info,party aff,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,venue
0,238.json,False,"When Obama was sworn into office, he DID NOT u...","obama-birth-certificate,religion",chain-email,NaN,NaN,none,11,43,8,5,105,NaN
1,7891.json,False,Says Having organizations parading as being so...,"campaign-finance,congress,taxes",earl-blumenauer,U.S. representative,Oregon,democrat,0,1,1,1,0,a U.S. Ways and Means hearing
2,8169.json,True,Says nearly half of Oregons children are poor.,poverty,jim-francesconi,Member of the State Board of Higher Education,Oregon,none,0,1,1,1,0,an opinion article
3,929.json,True,On attacks by Republicans that various program...,"economy,stimulus",barack-obama,President,Illinois,democrat,70,71,160,163,9,interview with CBS News
4,9416.json,False,Says when armed civilians stop mass shootings ...,guns,jim-rubens,Small business owner,New Hampshire,republican,1,1,0,1,0,"in an interview at gun shop in Hudson, N.H."


In [164]:
train_set['binary_label'] = train_set.label.apply(lambda x: binary_dict[x] )
val['binary_label'] = val.label.apply(lambda x: binary_dict[x] )
test['binary_label'] = test.label.apply(lambda x: binary_dict[x] )

In [165]:
x_train = preprocess(train_set['statement'])
y_train = train_set['binary_label']

x_val = preprocess(val['statement'])
y_val = val['binary_label']


#y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
x_test = preprocess(test['statement'])

In [167]:
!pip install tensorflow
!pip install keras
import tensorflow as tf
from keras.layers import Dense
#tf.keras.preprocessing.sequence.pad_sequences(seq)

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(list(x_train) +list(x_val)+ list(x_test))

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
x_val = tokenizer.texts_to_sequences(x_val)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=MAX_LEN)
x_val = tf.keras.preprocessing.sequence.pad_sequences(x_val,maxlen=MAX_LEN)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [168]:
max_features=None
max_features = max_features or len(tokenizer.word_index) + 1
max_features

13281

In [169]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [170]:
GLOVE_EMBEDDING_PATH = '/content/drive/MyDrive/glove.840B.300d.txt'


In [171]:
glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index,GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

n unknown words (glove):  1396


In [172]:
embedding_matrix = glove_matrix

In [173]:
x_train_torch = torch.tensor(x_train, dtype=torch.long).cuda()
x_test_torch = torch.tensor(x_test, dtype=torch.long).cuda()
y_train_torch = torch.tensor(y_train, dtype=torch.float32).cuda()
x_val_torch = torch.tensor(x_val , dtype=torch.long).cuda()
y_val_torch = torch.tensor(y_val, dtype=torch.float32).cuda()

In [174]:
train_dataset = data.TensorDataset(x_train_torch, y_train_torch)
test_dataset = data.TensorDataset(x_test_torch)
val_dataset = data.TensorDataset(x_val_torch,y_val_torch)

all_test_preds = []

model = NeuralNet(embedding_matrix)
model.cuda()

test_preds = train_model(model, train_dataset, val_dataset ,test_dataset, output_dim=1,
                         loss_fn=nn.BCEWithLogitsLoss(reduction='mean'),n_epochs=10)
all_test_preds.append(test_preds)

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1/10 	 loss=0.0990 	 time=12.23s
Epoch 2/10 	 loss=0.0998 	 time=11.29s
Epoch 3/10 	 loss=0.0975 	 time=11.23s
Epoch 4/10 	 loss=0.1009 	 time=10.96s
Epoch 5/10 	 loss=0.0970 	 time=10.85s
Epoch 6/10 	 loss=0.0986 	 time=10.82s
Epoch 7/10 	 loss=0.0988 	 time=10.86s
Epoch 8/10 	 loss=0.0997 	 time=10.88s
Epoch 9/10 	 loss=0.0992 	 time=10.97s
Epoch 10/10 	 loss=0.0992 	 time=10.94s


In [175]:
from sklearn.metrics import accuracy_score
accuracy_score(test['binary_label'],np.where((np.mean(all_test_preds, axis=0)[:, 0])>0.5,1,0))

0.6161137440758294